<a href="https://colab.research.google.com/github/infiniteoverflow/Intel-Image-Classification-Challenge/blob/master/Image_Scene_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

# Upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aswin1871","key":"d235db9386d12e7257e0e0ba61b54cf8"}'}

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [3]:
!kaggle datasets download -d puneet6060/intel-image-classification

 99% 343M/346M [00:07<00:00, 35.0MB/s]
100% 346M/346M [00:08<00:00, 45.2MB/s]


In [ ]:
!unzip intel-image-classification.zip -d dataset

In [6]:
import glob
import numpy as np
import matplotlib.pyplot as plt


In [7]:
glob.glob('/content/dataset/seg_train/seg_train/*')

['/content/dataset/seg_train/seg_train/buildings',
 '/content/dataset/seg_train/seg_train/glacier',
 '/content/dataset/seg_train/seg_train/mountain',
 '/content/dataset/seg_train/seg_train/sea',
 '/content/dataset/seg_train/seg_train/forest',
 '/content/dataset/seg_train/seg_train/street']

In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
datagen = ImageDataGenerator(rescale=1./255)

In [26]:
train_generator = datagen.flow_from_directory(
    directory = r"/content/dataset/seg_train/seg_train/",
    target_size=(150,150),
)

Found 14034 images belonging to 6 classes.


In [27]:
type(train_generator)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [28]:
test_generator = datagen.flow_from_directory(
    directory = r'/content/dataset/seg_test/seg_test/',
    target_size = (150,150),
)

Found 3000 images belonging to 6 classes.


## InceptionV3

In [29]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [34]:
base_model = InceptionV3(include_top=False,weights='imagenet')

In [35]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [36]:
for layer in base_model.layers:
  layer.trainable = False

In [37]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [39]:
x = base_model.output

x = tf.keras.layers.Dense(1024,activation='relu')(x)
predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

In [40]:
model = tf.keras.Model(inputs=base_model.input,outputs=predictions)

In [41]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________